In [1]:
from qdrant_client import QdrantClient, models

/home/berk/finreg-navigator/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = QdrantClient(url="http://localhost:6333")

In [3]:
collection_name = "bank_compliance_v1"

client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
            hnsw_config=models.HnswConfigDiff(
                m=8,
                ef_construct=100
            )
        )
    },
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    },
    optimizers_config=models.OptimizersConfigDiff(
        indexing_threshold=1000
    )
)

client.create_payload_index(collection_name, "file_name", models.PayloadSchemaType.KEYWORD)
client.create_payload_index(collection_name, "page_label", models.PayloadSchemaType.INTEGER)
client.create_payload_index(collection_name, "article_no", models.PayloadSchemaType.KEYWORD)

UpdateResult(operation_id=6, status=<UpdateStatus.COMPLETED: 'completed'>)

In [4]:
import os
from llama_index.core import SimpleDirectoryReader

def banking_metadata_extractor(file_path):
    """
    Her dosya okunduğunda dosya bazlı sabit metadata'ları ekler.
    LlamaIndex sayfa numaralarını (page_label) otomatik olarak dökümana ekleyecektir.
    """
    file_name = os.path.basename(file_path)
    
    # Dosya ismine göre döküman başlığı atama
    doc_titles = {
        "BBSEBH.pdf": "Bankaların Bilgi Sistemleri ve Elektronik Bankacılık Hizmetleri Hakkında Yönetmelik",
        "sir_saklama_yukumlulugu.pdf": "Sır Niteliğindeki Bilgilerin Paylaşılması Hakkında Yönetmelik"
    }
    
    return {
        "file_name": file_name,
        "document_title": doc_titles.get(file_name, "Bilinmeyen Mevzuat"),
        "file_path": file_path,
        "category": "Banking Regulation",
        "ingestion_date": "2024-05-22"
    }

# Veriyi Yükleme
reader = SimpleDirectoryReader(
    input_dir="./data", 
    file_metadata=banking_metadata_extractor
)
documents = reader.load_data()

print(documents[0].metadata)

{'page_label': '1', 'file_name': 'BBSEBH.pdf', 'document_title': 'Bankaların Bilgi Sistemleri ve Elektronik Bankacılık Hizmetleri Hakkında Yönetmelik', 'file_path': '/home/berk/finreg-navigator/data/BBSEBH.pdf', 'category': 'Banking Regulation', 'ingestion_date': '2024-05-22'}


In [5]:
from llama_index.core.node_parser import HierarchicalNodeParser

node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048,512,128],
    chunk_overlap=20
)

nodes = node_parser.get_nodes_from_documents(documents)

In [ ]:
import re

def enrich_metadata_with_inheritance(nodes):
    """
    Her node'un ebeveyninden madde numarasını miras almasını sağlar.
    """
    node_dict = {n.node_id: n for n in nodes}
    
    for node in nodes:
        match = re.search(r"MADDE\s+(\d+)", node.text)
        if match:
            node.metadata["article_no"] = f"Madde {match.group(1)}"
        else:
            parent_id = node.parent_node.node_id if node.parent_node else None
            while parent_id and "article_no" not in node.metadata:
                parent = node_dict.get(parent_id)
                if parent:
                    parent_match = re.search(r"MADDE\s+(\d+)", parent.text)
                    if parent_match:
                        node.metadata["article_no"] = f"Madde {parent_match.group(1)}"
                        break
                    parent_id = parent.parent_node.node_id if parent.parent_node else None
                else:
                    break
            
            if "article_no" not in node.metadata:
                node.metadata["article_no"] = "Genel Hükümler"
                
    return nodes

enriched_nodes = enrich_metadata_with_inheritance(nodes)

In [8]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Banking_RAG_Ingestion")

with mlflow.start_run(run_name="Initial_Ingestion_Hybrid"):
    mlflow.log_param("chunk_sizes", "[2048, 512, 128]")
    mlflow.log_param("vector_store", "Qdrant")
    mlflow.log_param("collection_name", "banking_compliance_v1")
    
    mlflow.log_metric("total_nodes", len(enriched_nodes))
    
    print(f"Toplam {len(enriched_nodes)} node oluşturuldu ve MLflow'a loglandı.")

2026/02/20 23:18:21 INFO mlflow.tracking.fluent: Experiment with name 'Banking_RAG_Ingestion' does not exist. Creating a new experiment.


Toplam 1751 node oluşturuldu ve MLflow'a loglandı.
🏃 View run Initial_Ingestion_Hybrid at: http://localhost:5000/#/experiments/1/runs/4cb7a4b627444b88b5215c302bd82d05
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [ ]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 1. Embedding Modelini Tanımla (Dense)
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 2. Qdrant Store Yapılandırması
vector_store = QdrantVectorStore(
    collection_name="bank_compliance_v1",
    client=client,
    enable_hybrid=True,
    batch_size=64,
    dense_vector_name="dense",
    sparse_vector_name="sparse"
)

# 3. Indexleme İşlemi
#storage_context = StorageContext.from_defaults(vector_store=vector_store)
#index = VectorStoreIndex(
#    enriched_nodes, 
#    storage_context=storage_context,
#    embed_model=embed_model
#)

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model
)

print("İndeksleme başarıyla tamamlandı!")

2026-02-20 23:18:31,175 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2026-02-20 23:18:31,479 - INFO - HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
2026-02-20 23:18:31,480 - WARNING - Warning: You are sending unauthenticated requests to the HF Hub. Please set a HF_TOKEN to enable higher rate limits and faster downloads.
2026-02-20 23:18:31,524 - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json "HTTP/1.1 200 OK"
2026-02-20 23:18:31,671 - INFO - HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json "HTTP/1.1 307 Temporary Redirect"
2026-02-20 23:18:31,715 - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v

İndeksleme başarıyla tamamlandı!


In [10]:
retriever = index.as_retriever(similarity_top_k=3)

# Test Sorgusu
query = "Açık bankacılık servisleri nelerdir ve kimler erişebilir?"
retrieved_nodes = retriever.retrieve(query)

for i, node in enumerate(retrieved_nodes):
    print(f"\n--- Sonuç {i+1} (Skor: {node.score:.4f}) ---")
    print(f"Kaynak: {node.metadata.get('file_name')} | Sayfa: {node.metadata.get('page_label')}")
    print(f"Madde: {node.metadata.get('article_no', 'Belirtilmemiş')}")
    print(f"İçerik Özeti: {node.text[:200]}...")

2026-02-20 23:22:57,545 - INFO - HTTP Request: POST http://localhost:6333/collections/bank_compliance_v1/points/query/batch "HTTP/1.1 200 OK"



--- Sonuç 1 (Skor: 0.7274) ---
Kaynak: BBSEBH.pdf | Sayfa: 1
Madde: Madde 1
İçerik Özeti: n) Güvenlik duvarı: Farklı güvenlik seviyelerine sahip ağlar veya ağa bağlı cihazlar arasındaki trafik akış kontrolünü sağlayan donanım ya da
yazılımları,
o) Hassas veri: Kimlik doğrulamada kullanılan...

--- Sonuç 2 (Skor: 0.7257) ---
Kaynak: BBSEBH.pdf | Sayfa: 2
Madde: Madde 4
İçerik Özeti: bilgi sistemlerinden sorumlu üst düzey yönetici ile bankanın ilgili iş
birimlerinden üst düzey yöneticilerin bu komiteye üye olması esastır....

--- Sonuç 3 (Skor: 0.7224) ---
Kaynak: sir_saklama_yukumlulugu.pdf | Sayfa: 1
Madde: Madde 1
İçerik Özeti: c) Ana sermaye: 5/9/2013 tarihli ve 28756 sayılı Resmî Gazete’de yayımlanan Bankaların Özkaynaklarına
İlişkin Yönetmelikte belirlenen usul ve esaslar çerçevesinde hesaplanacak ana sermayeyi,
ç) Anonim...


In [25]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3", request_timeout=120.0)

#response = llm.complete("Bankacılıkta uyum (compliance) ne anlama gelir?")
#print(response)

In [29]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import LLMRerank

# 1. Önce veritabanındaki tüm node ilişkilerini alıyoruz
all_nodes = enriched_nodes

# 2. Recursive Retriever tanımlama
base_retriever = index.as_retriever(similarity_top_k=20)

reranker = LLMRerank(choice_batch_size=10, top_n=5, llm=llm)

retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": base_retriever},
    node_dict={n.node_id: n for n in all_nodes},
    verbose=True
)

# 3. Query Engine oluşturma (LLM + Retriever birleşimi)
query_engine = RetrieverQueryEngine.from_args(
    retriever, 
    llm=llm,
    node_postprocessors=[reranker]
)

In [30]:
from llama_index.core import PromptTemplate

qa_prompt_tmpl_str = (
    "SADECE SANA VERİLEN BAĞLAMDAKİ BİLGİLERİ KULLANARAK CEVAP VER.\n"
    "KESİNLİKLE TÜRKÇE DIŞINDA BİR DİL KULLANMA.\n"
    "Sen kıdemli bir Bankacılık Uyum (Compliance) Analistisin.\n"
    "Cevabını verirken mutlaka hangi maddeye dayandığını belirt (Örn: Madde 3 uyarınca...).\n"
    "Eğer bağlamda bilgi yoksa, 'Bu bilgi dökümanda yer almamaktadır' de.\n"
    "---------------------\n"
    "BAĞLAM:\n{context_str}\n"
    "---------------------\n"
    "SORU: {query_str}\n"
    "YANIT (SADECE TÜRKÇE): "
)

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [31]:
response = query_engine.query("Açık bankacılık servislerine kimler erişebilir ve şartları nelerdir?")

print(f"Cevap: \n{response}")

for source in response.source_nodes:
    print(f"\nKullanılan Madde: {source.node.metadata.get('article_no')}")
    print(f"Kaynak Dosya: {source.node.metadata.get('file_name')}")

2026-02-20 23:41:04,133 - INFO - HTTP Request: POST http://localhost:6333/collections/bank_compliance_v1/points/query/batch "HTTP/1.1 200 OK"


Retrieving with query id None: Açık bankacılık servislerine kimler erişebilir ve şartları nelerdir?
Retrieving text node: bilgi sistemlerinden sorumlu üst düzey yönetici ile bankanın ilgili iş
birimlerinden üst düzey yöneticilerin bu komiteye üye olması esastır.
Retrieving text node: n) Güvenlik duvarı: Farklı güvenlik seviyelerine sahip ağlar veya ağa bağlı cihazlar arasındaki trafik akış kontrolünü sağlayan donanım ya da
yazılımları,
o) Hassas veri: Kimlik doğrulamada kullanılan veriler başta olmak üzere; müşteriye ait olan, çeşitli sebeplerle bankaca muhafaza edilen ve
üçüncü kişilerce ele geçirilmesi halinde, bu kişilerin müşteri olan kişilerle ayırt edilebilme mekanizmalarının zarar göreceği ve dolandırıcılık ya da
müşteriler adına sahte işlem yapılmasına imkân verebilecek nitelikteki verileri,
ö) İkincil merkez: İkincil sistemlerin kullanıma hazır olacak şekilde tesis edildiği ve birincil sistemlerde herhangi bir kesinti yaşanması durumunda
personelin çalışmasına imkân tanıyacak 

2026-02-20 23:41:27,306 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
2026-02-20 23:41:51,117 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
2026-02-20 23:41:55,926 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Cevap: 
Madde 17 uyarınca, Bilgi Sistemleri Süreklilik Planının ya da ikincil merkezin devreye alınması gibi hallerde derhal Sektörel SOME'yi bilgilendirir. Bu bilgi dökümanda yer almamaktadır.

Kullanılan Madde: Madde 29
Kaynak Dosya: BBSEBH.pdf

Kullanılan Madde: Madde 17
Kaynak Dosya: BBSEBH.pdf
